In [2]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import json
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from spdnetwork.optimizers import  MixOptimizer 
from spdnetwork.nn import LogEig
from Utils import get_fold_of_data
from DatasetManagement import DatasetManagement
from Models import Contrastive_CB3, SPDnet, SPDnet1Bire
from spdnetwork.optimizers import MixOptimizer


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["OMP_NUM_THREADS"] = str(1)

%load_ext autoreload
%autoreload 2

In [13]:
fold = 5
train_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/train/embeddingsSPD_train.pt'
train_labels_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/train/labelsSPD_train.pt'
val_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/embeddingsSPD_val.pt'
val_labels_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/labelsSPD_val.pt'

train_embeddings = torch.load(train_embeddings_path)
train_labels = torch.load(train_labels_path)
val_embeddings = torch.load(val_embeddings_path)
val_labels = torch.load(val_labels_path)

print(train_embeddings.shape, train_labels.shape, val_embeddings.shape, val_labels.shape)

torch.Size([1043, 96, 96]) torch.Size([1043, 1]) torch.Size([252, 96, 96]) torch.Size([252, 1])


In [26]:
def load_data(fold):
    train_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsSPD/fold_{fold}/train/embeddingsSPD_train.pt'
    train_labels_path = f'/data/ProstateContrastiveSPD/embeddingsSPD/fold_{fold}/train/labelsSPD_train.pt'
    val_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsSPD/fold_{fold}/val/embeddingsSPD_val.pt'
    val_labels_path = f'/data/ProstateContrastiveSPD/embeddingsSPD/fold_{fold}/val/labelsSPD_val.pt'
    
    train_embeddings = torch.load(train_embeddings_path)
    train_labels = torch.load(train_labels_path).view(-1)
    val_embeddings = torch.load(val_embeddings_path)
    val_labels = torch.load(val_labels_path).view(-1)
    
    return train_embeddings, train_labels, val_embeddings, val_labels

# Configuración de entrenamiento
num_folds = 5
batch_size = 32
num_epochs = 80
lr = 0.001
lr_others = 0.0001
momentum = 0.6

save_path = '/data/ProstateContrastiveSPD/ResultsPercentageNTXENT/' 
percentages = [0.2, 0.4, 0.6, 0.8]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

accuracy_list = []
auc_list = []

# Entrenamiento y validación en cada fold
for fold in range(1, num_folds + 1):
    print(f'Starting fold {fold}')
    
    # Cargar datos
    train_embeddings, train_labels, val_embeddings, val_labels = load_data(fold)
    
    total_train_samples = len(train_embeddings)
    
    for percentage in percentages:
        print(f'Training with {int(percentage * 100)}% of data')
        
        subset_size = int(total_train_samples * percentage)
        subset_train_embeddings = train_embeddings[:subset_size]
        subset_train_labels = train_labels[:subset_size]
        
        train_dataset = TensorDataset(subset_train_embeddings, subset_train_labels)
        val_dataset = TensorDataset(val_embeddings, val_labels)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
        
        model = SPDnet(device).to(device)
        print("------------Entrenando con la SPDnet------------")
        
        criterion = nn.CrossEntropyLoss()
        optimizer_class = torch.optim.RMSprop
        optimizer = MixOptimizer(parameters=model.parameters(),    
                                optimizer=optimizer_class,                                
                                lr=lr,
                                lr_others=lr_others,
                                momentum=momentum                               
                            )
        
        best_auc = 0
        best_model = None
        
        train_losses = []
        val_losses = []
        
        # Entrenamiento
        for epoch in range(num_epochs):
            model.train()
            running_train_loss = 0.0
            
            for batch_embeddings, batch_labels in train_loader:
                batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
                optimizer.zero_grad()
                batch_embeddings = batch_embeddings.unsqueeze(1)  # Ajustar el tamaño del batch para el modelo
                outputs = model(batch_embeddings)
                loss = criterion(outputs, batch_labels.long())
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item()
                
            train_loss = running_train_loss / len(train_loader)
            train_losses.append(train_loss)
        
            # Validación
            model.eval()
            running_val_loss = 0.0

            all_preds = []
            all_labels = []
        
            with torch.no_grad():
                    
                model.eval()
                model.to(device)
                for batch_embeddings, batch_labels in val_loader:
                    batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
                    batch_embeddings = batch_embeddings.unsqueeze(1)  # Ajustar el tamaño del batch para el modelo
                    outputs = model(batch_embeddings)
                    loss = criterion(outputs, batch_labels.long())
                    running_val_loss += loss.item()
                    
                    probs = torch.softmax(outputs, dim=1)  # Usar softmax para obtener probabilidades de clases
                    preds = torch.argmax(probs, dim=1).cpu().numpy()  # Obtener las predicciones de clase
                    all_preds.extend(preds)
                    all_labels.extend(batch_labels.cpu().numpy())
        
            val_loss = running_val_loss / len(val_loader)
            val_losses.append(val_loss)
            
            accuracy = accuracy_score(all_labels, all_preds)
            auc = roc_auc_score(all_labels, all_preds)
        
            if auc > best_auc:
                best_auc = auc
                best_model = model.state_dict()
            
            print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}, Val AUC: {auc:.4f}')
        
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        
        print(f'Fold {fold}, {int(percentage * 100)}% Data, Accuracy: {accuracy}, AUC: {auc}')

        # Crear la carpeta para guardar el modelo del pliegue correspondiente
        fold_save_path = os.path.join(save_path, f'fold_{fold}', f'{int(percentage * 100)}%')
        os.makedirs(fold_save_path, exist_ok=True)
        
        # Guardar el mejor modelo
        best_model_path = os.path.join(fold_save_path, 'best_model.pth')
        torch.save(best_model, best_model_path)
        
        plt.figure()
        plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
        plt.plot(range(1, num_epochs + 1), val_losses, label='Val Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Fold {fold} - {int(percentage * 100)}% Data Loss')
        plt.legend()
        plt.savefig(os.path.join(fold_save_path, 'loss_plot.png'))
        plt.close()

# Resultados promedio
print(f'Mean Accuracy: {sum(accuracy_list) / (num_folds * len(percentages))}')
print(f'Mean AUC: {sum(auc_list) / (num_folds * len(percentages))}')


Using device: cuda
Starting fold 1
Training with 20% of data
------------Entrenando con la SPDnet------------
Epoch 1/80, Train Loss: 0.4583, Val Loss: 0.5076, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 2/80, Train Loss: 0.4123, Val Loss: 0.4604, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 3/80, Train Loss: 0.3873, Val Loss: 0.4555, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 4/80, Train Loss: 0.3680, Val Loss: 0.4527, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 5/80, Train Loss: 0.3859, Val Loss: 0.4480, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 6/80, Train Loss: 0.3588, Val Loss: 0.4479, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 7/80, Train Loss: 0.3504, Val Loss: 0.4535, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 8/80, Train Loss: 0.3282, Val Loss: 0.4545, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 9/80, Train Loss: 0.3332, Val Loss: 0.4544, Val Accuracy: 0.8242, Val AUC: 0.5194
Epoch 10/80, Train Loss: 0.3491, Val Loss: 0.4541, Val Accuracy: 0.8281, Val AUC: 0.5302
Epoch 11/

# Borrar

In [27]:
def load_data(fold):
    train_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/train/embeddingsSPD_train.pt'
    train_labels_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/train/labelsSPD_train.pt'
    val_embeddings_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/embeddingsSPD_val.pt'
    val_labels_path = f'/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/labelsSPD_val.pt'
    
    train_embeddings = torch.load(train_embeddings_path)
    train_labels = torch.load(train_labels_path).view(-1)
    val_embeddings = torch.load(val_embeddings_path)
    val_labels = torch.load(val_labels_path).view(-1)
    
    return train_embeddings, train_labels, val_embeddings, val_labels

# Configuración de entrenamiento
num_folds = 5
batch_size = 32
num_epochs = 80
lr = 0.001
lr_others = 0.0001
momentum = 0.6

save_path = '/data/ProstateContrastiveSPD/ResultsPercentageTriplet/' 
percentages = [0.2, 0.4, 0.6, 0.8]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

accuracy_list = []
auc_list = []

# Entrenamiento y validación en cada fold
for fold in range(1, num_folds + 1):
    print(f'Starting fold {fold}')
    
    # Cargar datos
    train_embeddings, train_labels, val_embeddings, val_labels = load_data(fold)
    
    total_train_samples = len(train_embeddings)
    
    for percentage in percentages:
        print(f'Training with {int(percentage * 100)}% of data')
        
        subset_size = int(total_train_samples * percentage)
        subset_train_embeddings = train_embeddings[:subset_size]
        subset_train_labels = train_labels[:subset_size]
        
        train_dataset = TensorDataset(subset_train_embeddings, subset_train_labels)
        val_dataset = TensorDataset(val_embeddings, val_labels)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
        
        model = SPDnet(device).to(device)
        print("------------Entrenando con la SPDnet------------")
        
        criterion = nn.CrossEntropyLoss()
        optimizer_class = torch.optim.RMSprop
        optimizer = MixOptimizer(parameters=model.parameters(),    
                                optimizer=optimizer_class,                                
                                lr=lr,
                                lr_others=lr_others,
                                momentum=momentum                               
                            )
        
        best_auc = 0
        best_model = None
        
        train_losses = []
        val_losses = []
        
        # Entrenamiento
        for epoch in range(num_epochs):
            model.train()
            running_train_loss = 0.0
            
            for batch_embeddings, batch_labels in train_loader:
                batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
                optimizer.zero_grad()
                batch_embeddings = batch_embeddings.unsqueeze(1)  # Ajustar el tamaño del batch para el modelo
                outputs = model(batch_embeddings)
                loss = criterion(outputs, batch_labels.long())
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item()
                
            train_loss = running_train_loss / len(train_loader)
            train_losses.append(train_loss)
        
            # Validación
            model.eval()
            running_val_loss = 0.0

            all_preds = []
            all_labels = []
        
            with torch.no_grad():
                    
                model.eval()
                model.to(device)
                for batch_embeddings, batch_labels in val_loader:
                    batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
                    batch_embeddings = batch_embeddings.unsqueeze(1)  # Ajustar el tamaño del batch para el modelo
                    outputs = model(batch_embeddings)
                    loss = criterion(outputs, batch_labels.long())
                    running_val_loss += loss.item()
                    
                    probs = torch.softmax(outputs, dim=1)  # Usar softmax para obtener probabilidades de clases
                    preds = torch.argmax(probs, dim=1).cpu().numpy()  # Obtener las predicciones de clase
                    all_preds.extend(preds)
                    all_labels.extend(batch_labels.cpu().numpy())
        
            val_loss = running_val_loss / len(val_loader)
            val_losses.append(val_loss)
            
            accuracy = accuracy_score(all_labels, all_preds)
            auc = roc_auc_score(all_labels, all_preds)
        
            if auc > best_auc:
                best_auc = auc
                best_model = model.state_dict()
            
            print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.4f}, Val AUC: {auc:.4f}')
        
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        
        print(f'Fold {fold}, {int(percentage * 100)}% Data, Accuracy: {accuracy}, AUC: {auc}')

        # Crear la carpeta para guardar el modelo del pliegue correspondiente
        fold_save_path = os.path.join(save_path, f'fold_{fold}', f'{int(percentage * 100)}%')
        os.makedirs(fold_save_path, exist_ok=True)
        
        # Guardar el mejor modelo
        best_model_path = os.path.join(fold_save_path, 'best_model.pth')
        torch.save(best_model, best_model_path)
        
        plt.figure()
        plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
        plt.plot(range(1, num_epochs + 1), val_losses, label='Val Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Fold {fold} - {int(percentage * 100)}% Data Loss')
        plt.legend()
        plt.savefig(os.path.join(fold_save_path, 'loss_plot.png'))
        plt.close()

# Resultados promedio
print(f'Mean Accuracy: {sum(accuracy_list) / (num_folds * len(percentages))}')
print(f'Mean AUC: {sum(auc_list) / (num_folds * len(percentages))}')


Using device: cuda
Starting fold 1
Training with 20% of data
------------Entrenando con la SPDnet------------
Epoch 1/80, Train Loss: 0.5810, Val Loss: 0.4664, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 2/80, Train Loss: 0.4171, Val Loss: 0.4692, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 3/80, Train Loss: 0.4093, Val Loss: 0.4627, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 4/80, Train Loss: 0.3695, Val Loss: 0.4633, Val Accuracy: 0.8203, Val AUC: 0.5000
Epoch 5/80, Train Loss: 0.4077, Val Loss: 0.4615, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 6/80, Train Loss: 0.3588, Val Loss: 0.4661, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 7/80, Train Loss: 0.3902, Val Loss: 0.4641, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 8/80, Train Loss: 0.3773, Val Loss: 0.4660, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 9/80, Train Loss: 0.3767, Val Loss: 0.4698, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 10/80, Train Loss: 0.3626, Val Loss: 0.4724, Val Accuracy: 0.8164, Val AUC: 0.4976
Epoch 11/

# --------------------------------------------------------


# ================ Métricas ================

# --------------------------------------------------------

In [34]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve, auc

import numpy as np

# Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
list_auc = []
list_accuracy = []
list_aucpr = []
# Definir una función para cargar los datos y etiquetas de validación
def load_data(embeddings_path, labels_path):
    embeddings = torch.load(embeddings_path, map_location=device)
    labels = torch.load(labels_path, map_location=device)
    return embeddings, labels

# Definir una función para evaluar el modelo y calcular métricas
def evaluate_model(model, val_embeddings, val_labels):
    model.eval()
    with torch.no_grad():
        # Obtener las predicciones del modelo
        outputs = model(val_embeddings)
        # Asumiendo que las salidas son logits, aplicar softmax para obtener probabilidades
        probs = F.softmax(outputs, dim=1)[:, 1]  # Suponiendo que la clase positiva es la 1
        preds = torch.argmax(outputs, dim=1)
        
        # Calcular AUC-ROC
        auc_roc = roc_auc_score(val_labels.cpu().numpy(), probs.cpu().numpy())
        
        # Calcular Accuracy
        accuracy = accuracy_score(val_labels.cpu().numpy(), preds.cpu().numpy())
        
        #Calcular AUCPR
        precisions, recalls, _ = precision_recall_curve(val_labels.cpu().numpy(), probs.cpu().numpy())
        auc_pr = auc(recalls, precisions)
        
        return auc_roc, accuracy, auc_pr

# Path base para los datos
base_path = "/data/ProstateContrastiveSPD/ResultsPercentageTriplet/"

folds = [1,2,3,4,5]  # Puedes agregar más folds aquí

for fold in folds:
    # Cargar el modelo para el fold actual
    model_path = f"{base_path}/fold_{fold}/60%/best_model.pth"
    model = SPDnet(device).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    
    # Cargar los datos de validación
    val_embeddings_path = f"/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/embeddingsSPD_val.pt"
    val_labels_path = f"/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/labelsSPD_val.pt"
    
    val_embeddings, val_labels = load_data(val_embeddings_path, val_labels_path)
    val_embeddings = val_embeddings.unsqueeze(1)
    
    # Evaluar el modelo y calcular métricas
    auc_roc, accuracy, auc_pr = evaluate_model(model, val_embeddings, val_labels)
    list_auc.append(auc_roc)
    list_accuracy.append(accuracy)
    list_aucpr.append(auc_pr)
    
    print(f"Fold {fold} - AUC-ROC: {auc_roc:.4f}, Accuracy: {accuracy:.4f}")
print(f"Promedio de AUC: {np.mean(list_auc)}")
print(f"Desviación estandar: {np.std(list_auc)}")

print(f"Promedio de Accuracy: {np.mean(list_accuracy)}")
print(f"Desviación estandar: {np.std(list_accuracy)}")

print(f"Promedio de AUC_PR: {np.mean(list_aucpr)}")
print(f"Desviación estandar: {np.std(list_aucpr)}")


Fold 1 - AUC-ROC: 0.8898, Accuracy: 0.8731
Fold 2 - AUC-ROC: 0.8862, Accuracy: 0.8295
Fold 3 - AUC-ROC: 0.8576, Accuracy: 0.8740
Fold 4 - AUC-ROC: 0.9178, Accuracy: 0.9057
Fold 5 - AUC-ROC: 0.8899, Accuracy: 0.8849
Promedio de AUC: 0.8882851463807361
Desviación estandar: 0.019100971706590534
Promedio de Accuracy: 0.8734438275865998
Desviación estandar: 0.02488835893891493
Promedio de AUC_PR: 0.7022977327471926
Desviación estandar: 0.040102342537969835


# Por porcentajes

In [32]:
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve, auc
import numpy as np

# Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definir una función para cargar los datos y etiquetas de validación
def load_data(embeddings_path, labels_path):
    embeddings = torch.load(embeddings_path, map_location=device)
    labels = torch.load(labels_path, map_location=device)
    return embeddings, labels

# Definir una función para evaluar el modelo y calcular métricas
def evaluate_model(model, val_embeddings, val_labels):
    model.eval()
    with torch.no_grad():
        # Obtener las predicciones del modelo
        outputs = model(val_embeddings)
        # Asumiendo que las salidas son logits, aplicar softmax para obtener probabilidades
        probs = F.softmax(outputs, dim=1)[:, 1]  # Suponiendo que la clase positiva es la 1
        preds = torch.argmax(outputs, dim=1)
        
        # Calcular AUC-ROC
        auc_roc = roc_auc_score(val_labels.cpu().numpy(), probs.cpu().numpy())
        
        # Calcular Accuracy
        accuracy = accuracy_score(val_labels.cpu().numpy(), preds.cpu().numpy())
        
        #Calcular AUCPR
        precisions, recalls, _ = precision_recall_curve(val_labels.cpu().numpy(), probs.cpu().numpy())
        auc_pr = auc(recalls, precisions)
        
        return auc_roc, accuracy, auc_pr

# Path base para los datos y modelos
base_path = "/data/ProstateContrastiveSPD/ResultsPercentageTriplet/"

folds = [1, 2, 3, 4, 5]  # Folds disponibles
percentages = ['20%','40%', '60%', '80%']  # Porcentajes de datos utilizados en el entrenamiento

# Diccionarios para almacenar los resultados de cada porcentaje
metrics = {percentage: {'auc': [], 'accuracy': [], 'aucpr': []} for percentage in percentages}

for fold in folds:
    for percentage in percentages:
        # Cargar el modelo para el fold y porcentaje actual
        model_path = f"{base_path}/fold_{fold}/{percentage}/best_model.pth"
        model = SPDnet(device).to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        
        # Cargar los datos de validación
        val_embeddings_path = f"/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/embeddingsSPD_val.pt"
        val_labels_path = f"/data/ProstateContrastiveSPD/embeddingsTripletSPD/fold_{fold}/val/labelsSPD_val.pt"
        
        val_embeddings, val_labels = load_data(val_embeddings_path, val_labels_path)
        val_embeddings = val_embeddings.unsqueeze(1)
        
        # Evaluar el modelo y calcular métricas
        auc_roc, accuracy, auc_pr = evaluate_model(model, val_embeddings, val_labels)
        
        # Almacenar las métricas en el diccionario correspondiente al porcentaje
        metrics[percentage]['auc'].append(auc_roc)
        metrics[percentage]['accuracy'].append(accuracy)
        metrics[percentage]['aucpr'].append(auc_pr)
        
        print(f"Fold {fold} - {percentage}% - AUC-ROC: {auc_roc:.4f}, Accuracy: {accuracy:.4f}, AUC-PR: {auc_pr:.4f}")

# Calcular y mostrar los promedios y desviaciones estándar para cada porcentaje
for percentage in percentages:
    mean_auc = np.mean(metrics[percentage]['auc'])
    std_auc = np.std(metrics[percentage]['auc'])
    mean_accuracy = np.mean(metrics[percentage]['accuracy'])
    std_accuracy = np.std(metrics[percentage]['accuracy'])
    mean_aucpr = np.mean(metrics[percentage]['aucpr'])
    std_aucpr = np.std(metrics[percentage]['aucpr'])
    
    print(f"\n{percentage}% de datos:")
    print(f"Promedio de AUC-ROC: {mean_auc:.4f}, Desviación estándar: {std_auc:.4f}")
    print(f"Promedio de Accuracy: {mean_accuracy:.4f}, Desviación estándar: {std_accuracy:.4f}")
    print(f"Promedio de AUC-PR: {mean_aucpr:.4f}, Desviación estándar: {std_aucpr:.4f}")


Fold 1 - 20%% - AUC-ROC: 0.8088, Accuracy: 0.8308, AUC-PR: 0.4709
Fold 1 - 40%% - AUC-ROC: 0.8587, Accuracy: 0.8885, AUC-PR: 0.6781
Fold 1 - 60%% - AUC-ROC: 0.8898, Accuracy: 0.8731, AUC-PR: 0.7012
Fold 1 - 80%% - AUC-ROC: 0.8676, Accuracy: 0.8692, AUC-PR: 0.6706
Fold 2 - 20%% - AUC-ROC: 0.8088, Accuracy: 0.8864, AUC-PR: 0.6513
Fold 2 - 40%% - AUC-ROC: 0.8838, Accuracy: 0.9015, AUC-PR: 0.7310
Fold 2 - 60%% - AUC-ROC: 0.8862, Accuracy: 0.8295, AUC-PR: 0.7370
Fold 2 - 80%% - AUC-ROC: 0.8981, Accuracy: 0.9129, AUC-PR: 0.7362
Fold 3 - 20%% - AUC-ROC: 0.8561, Accuracy: 0.8780, AUC-PR: 0.6515
Fold 3 - 40%% - AUC-ROC: 0.8609, Accuracy: 0.8780, AUC-PR: 0.6731
Fold 3 - 60%% - AUC-ROC: 0.8576, Accuracy: 0.8740, AUC-PR: 0.6369
Fold 3 - 80%% - AUC-ROC: 0.8537, Accuracy: 0.8780, AUC-PR: 0.6619
Fold 4 - 20%% - AUC-ROC: 0.8899, Accuracy: 0.8868, AUC-PR: 0.6581
Fold 4 - 40%% - AUC-ROC: 0.9184, Accuracy: 0.8943, AUC-PR: 0.7331
Fold 4 - 60%% - AUC-ROC: 0.9178, Accuracy: 0.9057, AUC-PR: 0.7502
Fold 4 - 8